# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Inference Pipeline</span>


## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [19]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

2025-11-21 13:52:05,830 INFO: Connection closed.
2025-11-21 13:52:05,939 INFO: Python Engine initialized.

Logged in to project, explore it here https://hopsworks.ai.local/p/120


## <span style="color:#ff5f27;"> ⚙️ Feature Group Retrieval</span>
Let's retrieve a feature group in order to get cc_num values.

In [20]:
# Retrieve the 'transactions_fraud_online_fg' feature group
trans_fg = fs.get_feature_group(
    'transactions_fraud_online_fg',
    version=1,
)

In [21]:
# Retrieve the first 5 unique credit card numbers (cc_nums)
cc_nums = trans_fg.select('cc_num').show(5).cc_num.values

# Display the obtained cc_nums
cc_nums

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.46s) 


array([4001468975474818, 4001468975474818, 4001468975474818,
       4001468975474818, 4001468975474818])

## <span style="color:#ff5f27;">🗄 Model Registry</span>


In [22]:
# Get the Model Registry
mr = project.get_model_registry()

## <span style='color:#ff5f27'>🚀 Fetch Deployment</span>

In [23]:
# Access the Model Serving
ms = project.get_model_serving()

# Specify the deployment name
deployment_name = "fraudonlinemodeldeployment"

# Get the deployment with the specified name
deployment = ms.get_deployment(deployment_name)

# Start the deployment and wait for it to be in a running state for up to 300 seconds
deployment.start(await_running=300)

Deployment is already running
Start making predictions by using `.predict()`


## <span style='color:#ff5f27'>🔮 Predicting using deployment</span>


Finally you can start making predictions with your model!

Send inference requests to the deployed model as follows:

In [24]:
# Get the first credit card number
cc_num = cc_nums[0]
cc_num

4001468975474818

In [25]:
# Make a prediction
deployment.predict(
    inputs=[int(cc_num)],
)

{'predictions': [0]}

In [26]:
# Predict for several cc_nums
predictions = [
    deployment.predict(inputs=[int(cc_num)])['predictions'] 
    for cc_num
    in cc_nums
]
predictions

[[0], [0], [0], [0], [0]]

## <span style='color:#ff5f27'>🔮 Read Prediction Log</span>

In [27]:
retrieved_model = mr.get_model(
    name="xgboost_fraud_online_model",
    version=1,
)
feature_view = retrieved_model.get_feature_view()

2025-11-21 13:52:30,074 INFO: Initializing for batch retrieval of feature vectors


In [28]:
# Stop the job materialization schedule and materialize log manually
feature_view.pause_logging()
feature_view.materialize_log(wait=True)

Launching job: transactions_fraud_online_fv_1_log_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://hopsworks.ai.local/p/120/jobs/named/transactions_fraud_online_fv_1_log_1_offline_fg_materialization/executions
2025-11-21 13:52:40,179 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-21 13:52:43,197 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-21 13:54:38,935 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-21 13:54:38,965 INFO: Waiting for log aggregation to finish.
2025-11-21 13:54:47,167 INFO: Execution finished successfully.


[Job('transactions_fraud_online_fv_1_log_1_offline_fg_materialization', 'PYSPARK')]

In [29]:
# Read untransformed log
feature_view.read_log().head(3)

2025-11-21 13:54:47,520 WARNING: DeprecationWarning: Providing ´transformed´ while fetching logging feature group is deprecated and will be dropped in future versions. Transformed and untransformed features are now logged in the same feature group.

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.28s) 


,request_id,td_version,model_name,model_version,request_parameters,cc_num,datetime,tid,amount,country,predicted_fraud_label,loc_delta_t_plus_1,loc_delta_t_minus_1,time_delta_t_minus_1,gender,label_encoder_country_,label_encoder_gender_
0,None,NaN,xgboost_fraud_online_model,1,None,4001468975474818,2022-04-14 15:10:27,7efd58d995554d554e70eaf028e566d6,316.44,US,0,0.500866,0.0,0.0,M,111,1
1,None,NaN,xgboost_fraud_online_model,1,None,4001468975474818,2022-04-14 15:10:27,7efd58d995554d554e70eaf028e566d6,316.44,US,0,0.500866,0.0,0.0,M,111,1
2,None,NaN,xgboost_fraud_online_model,1,None,4001468975474818,2022-04-14 15:10:27,7efd58d995554d554e70eaf028e566d6,316.44,US,0,0.500866,0.0,0.0,M,111,1


In [30]:
# Read transformed log
feature_view.read_log(transformed=True).head(3)

2025-11-21 13:54:52,688 WARNING: DeprecationWarning: Providing ´transformed´ while fetching logging feature group is deprecated and will be dropped in future versions. Transformed and untransformed features are now logged in the same feature group.

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.16s) 


,request_id,td_version,model_name,model_version,request_parameters,cc_num,datetime,tid,amount,country,predicted_fraud_label,loc_delta_t_plus_1,loc_delta_t_minus_1,time_delta_t_minus_1,gender,label_encoder_country_,label_encoder_gender_
0,None,NaN,xgboost_fraud_online_model,1,None,4001468975474818,2022-04-14 15:10:27,7efd58d995554d554e70eaf028e566d6,316.44,US,0,0.500866,0.0,0.0,M,111,1
1,None,NaN,xgboost_fraud_online_model,1,None,4001468975474818,2022-04-14 15:10:27,7efd58d995554d554e70eaf028e566d6,316.44,US,0,0.500866,0.0,0.0,M,111,1
2,None,NaN,xgboost_fraud_online_model,1,None,4001468975474818,2022-04-14 15:10:27,7efd58d995554d554e70eaf028e566d6,316.44,US,0,0.500866,0.0,0.0,M,111,1


### Stop Deployment
To stop the deployment you simply run:

In [ ]:
# Stop the deployment
deployment.stop(await_stopped=180)

## <span style="color:#ff5f27;">👾 StreamLit App</span>


If you want to see interactive dashboards - use a **StreamLit App**.

Type the next commands in terminal to run a Streamlit App:

`python -m streamlit run streamlit_app.py`

---

### <span style="color:#ff5f27;">🥳 <b> Next Steps  </b> </span>
Congratulations you've now completed the Fraud Online tutorial for Managed Hopsworks.

Check out our other tutorials on ➡ https://github.com/logicalclocks/hopsworks-tutorials

Or documentation at ➡ https://docs.hopsworks.ai